# TIPS

In [ ]:
CONVERT(Slot_begin, integer)

In [ ]:
( assets.`CONTAINER` LIKE '%uest' OR
   assets.`CONTAINER` = 'SUN zone' OR
   assets.`CONTAINER` = 'HP vpar' )

# Creation / Alteration de table

In [ ]:
CREATE TABLE assets_filtree_stockage_réseau
AS 
SELECT *
FROM `assets_filtree`
WHERE YPE IN ('networkcomponents','storage')

In [ ]:
Ajout des colonnes pour les coordonnees d'equipements
ALTER TABLE assets_filtree
ADD COLUMN STATUS varchar(255),
ADD COLUMN emplacement varchar(255),
ADD COLUMN Rack varchar(255),
ADD COLUMN Slot varchar(255),
ADD COLUMN Slot_begin varchar(255),
ADD COLUMN Slot_end varchar(255);

# VIEW

System 2-en-1 avec chassis + server. Y aura 2 objects differents alors que c'est la meme chose, on va donc s'assurer que le match se fait sur le chassis plutot que sur le server + on rattache tous les serveurs à au bon chassis

In [ ]:
CREATE VIEW it_built_system_storage_analytics
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment` it
WHERE  status IS NOT NULL
GROUP  BY nom_de_baie,
          nom_de_la_salle
ORDER  BY nom_de_baie DESC

Similaire à celle d'avant mais permet de classer tous les équipements par rack si un chassis est trouvé

In [ ]:
CREATE VIEW it_built_system_storage_analytics_by_rack
AS 
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
       it.Modele,
       it.reperage_ID,
       it.fabricant,
       it2.reperage_ID reperage_ID_chassis,
       it2.nom nom_chassis, 
       it2.Modele Modele_chassis,
       it2.fabricant fabricant_chassis
FROM   `IT_Equipment` it
LEFT JOIN `it_built_system_storage_analytics` it2 ON it2.nom_de_baie = it.nom_de_baie and it2.nom_de_la_salle = it.nom_de_la_salle
ORDER  BY nom_de_baie DESC

## Exadata

In [ ]:
CREATE VIEW it_build_system_EXADATA
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment` it
where it.fabricant like '%oracle%'

### NAS ( Netapp)

On regroupe tous les types NAS avec un asset_id pour faire correspondre les extensions aà l'equipment principal 

In [ ]:
CREATE VIEW it_built_system_NAS
AS
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment` it
WHERE  status IS NOT NULL and (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%' ) and ASSET_ID != '0'
ORDER  BY nom_de_baie DESC

# Hitachi

In [ ]:
CREATE VIEW it_build_system_NAS_Hitachi 
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment` it
where it.fabricant like '%Hitachi%'
ORDER BY Nom_de_baie DESC, Nom_de_la_salle

## HPC

In [ ]:
CREATE VIEW it_build_system_HPC
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID,
        (LEFT(Nom,6)) ROOT_FERME
FROM   `IT_Equipment` it
where it.nom like '%HPC%' or it.long_name like '%HPC%'
ORDER BY Nom_de_baie DESC, Nom_de_la_salle

In [ ]:
CREATE VIEW it_build_system_HPC
as
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
       (LEFT(it.Nom,6)) ROOT_FERME, 
        it.Modele,
        it.fabricant,
        it.Reperage_ID
       
FROM   `IT_Equipment` it
WHERE it.nom like '%HPC%' 
OR it.long_name like '%HPC%' 
AND it.status = 
			(CASE WHEN it.status in ('OK-CMDB-ASSET-ID','ASSET_ID OK / SHORT_NAME MISMATCH') then it.status
             ELSE it.status
             END)
GROUP BY ROOT_FERME
ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle

## HANA

In [ ]:
CREATE VIEW it_build_system_HANA
as
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
        it.Modele,
        it.fabricant,
        it.Reperage_ID,
        (CASE WHEN it.status in ('OK-CMDB-ASSET-ID','ASSET_ID OK / SHORT_NAME MISMATCH') then it.status
             ELSE it.status
             END) agg
       
FROM   `IT_Equipment` it
WHERE (it.nom_de_baie like '%hana%' 
OR it.nom like '%hana%') and (it.status is not null or it.asset_id != '0')	
GROUP BY nom_de_baie, nom_de_la_salle
ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle

## RING


Scality : systeme type "RING"

- Si le "Nom" contient "NA9TCR" attribuer sur la colonne "status" le nom "RING NA9TCR : M-0000655049"

- pour tout élément hors ( "status" = "OK-CMDB-ASSET-ID" OU "status" = "ASSET_ID OK / SHORT_NAME MISMATCH")

# Compteur

In [ ]:
SELECT count(*) count, TYPE
from `IT_Equipment`
where STATUS is null
GROUP BY TYPE

In [ ]:
SELECT count(*) count, YPE
from `assets_filtree_stockage_réseau`
where STATUS is null
GROUP BY YPE

In [ ]:
select COUNT(STATUS), status
from `assets_filtree`
GROUP BY STATUS
ORDER BY COUNT(STATUS) desc

In [ ]:
select COUNT(STATUS), status
from `IT_Equipment`
GROUP BY STATUS
ORDER BY COUNT(STATUS) desc

In [ ]:
select count(*)
from `IT_Equipment`
where status is null

# Query

## Query de base

In [ ]:
SELECT it.asset_id asset_id_id,
       it.short_name,
       it.nom,
       it.status   status_it,
       it.fabricant,
       it.modele,
       it.reperage_id,
       nom_de_la_salle,
       nom_de_baie,
       slot_u,
       hauteur_u
FROM   `it_equipment` it 

## Match sur les coordonnées

### Coordonnees par position de rack/baie 

In [ ]:
SELECT ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End
from `assets_filtree` aa
where dalle like '%L15%'
order by dalle asc

In [ ]:
SELECT it.asset_id asset_id_id,
       it.short_name,
       it.nom,
       it.status   status_it,
       it.fabricant,
       it.modele,
       it.reperage_id,
       nom_de_la_salle,
       nom_de_baie,
       slot_u,
       hauteur_u
FROM   `it_equipment` it 
where Reperage_ID like 'L15:%'
order by Reperage_ID asc

## Match sur les coordonnées inversées

In [ ]:
select it.asset_id asset_id_id, it.short_name, it.nom, it.status status_it, it.fabricant, it.modele, it.reperage_ID, it.nom_de_la_salle,
it.Nom_de_baie, SUBSTR(it.Nom_de_baie,length(it.Nom_de_baie),length(it.Nom_de_baie)) rp,
SUBSTR(it.Nom_de_baie,length(it.Nom_de_baie)-1,length(it.Nom_de_baie)) rp2
from `IT_Equipment` it
where STATUS is null
ORDER BY nom asc

----
SELECT ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End
from `assets_filtree` aa
where NOM_COURT like '%ch12%' or NOM_LONG like '%ch12%'
order by dalle asc


## Match sur les extensions : Fex / Netapp, Netapps

### Fex

In [ ]:
select it.asset_id asset_id_id, it.short_name, it.nom, it.status status_it, it.fabricant, it.modele, it.reperage_ID, Nom_de_la_salle, Nom_de_baie, Slot_U, Hauteur_U
from `IT_Equipment` it
where SHORT_NAME like '%SWC3PC27%' or nom like '%SWC3PC27%'

---

SELECT ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End
from `assets_filtree` aa
where NOM_COURT like '%SWC3PC27%' or NOM_LONG like '%SWC3PC27%'
order by dalle asc

## Match sur les Exadata

#### Aucun equipements ne possède d'asset ID pour changer le status..

In [ ]:
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment` it
where it.fabricant like '%oracle%'

## Match : Hitachi

In [ ]:
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment` it
where it.fabricant like '%Hitachi%'
ORDER BY Nom_de_baie DESC, Nom_de_la_salle

## HPC

In [ ]:
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID,
        (LEFT(Nom,6)) ROOT_FERME
FROM   `IT_Equipment` it
where it.nom like '%HPC%' or it.long_name like '%HPC%'
ORDER BY Nom_de_baie DESC, Nom_de_la_salle

In [ ]:
-- Visualisation de tous les equipment par rapport à leur nom commun
select it.asset_id asset_id_id, it.short_name, it.nom, (LEFT(it.Nom,6)) ROOT_FERME, it.status status_it, it.fabricant, it.modele, it.reperage_ID, Nom_de_la_salle, Nom_de_baie, Slot_U, Hauteur_U
from `IT_Equipment` it
where (LEFT(it.Nom,6)) like 'HPC1D3'
order by ROOT_FERME asc

In [ ]:
--- Maj par nom commun 
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
       (LEFT(it.Nom,6)) ROOT_FERME, hpc.root_ferme,
        it.Modele,
        it.fabricant,
        it.Reperage_ID,
       (CASE WHEN it.status not in ('OK-CMDB-ASSET-ID','ASSET_ID OK / SHORT_NAME MISMATCH') THEN Concat('HPC : ', hpc.root_ferme)
        WHEN it.status is null then Concat('HPC : ', hpc.root_ferme)
        WHEN it.status in ('OK-CMDB-ASSET-ID','ASSET_ID OK / SHORT_NAME MISMATCH') THEN it.status
        END ) new_status
        
FROM   `IT_Equipment` it
LEFT JOIN `it_build_system_HPC` hpc on (LEFT(it.Nom,6)) = hpc.root_ferme
WHERE it.nom like '%HPC%' 
OR it.long_name like '%HPC%'

## HANA

## RING

## NetApp

In [ ]:
SELECT ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, status
from `assets_filtree` aa
where fabricant like 'NETAPP%'
order by dalle asc

---
select it.asset_id asset_id_id, it.short_name, it.nom, it.status status_it, it.fabricant, it.modele, it.reperage_ID, Nom_de_la_salle, Nom_de_baie, Slot_U, Hauteur_U
from `IT_Equipment` it
where fabricant like 'NETAPP%'

## On regroupe tous les extensions sous le même cerveau pour que leur status corresponde au nom du cerveau

In [ ]:
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom
FROM   `it_equipment` it
WHERE  status IS NOT NULL
GROUP  BY nom_de_baie,
          nom_de_la_salle
ORDER  BY nom_de_baie DESC  

Join entre la vue qui contient les chassis et les extensions avec un status null

In [ ]:
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
       it.Modele,
       it2.nom, 
       it2.Modele
FROM   `IT_Equipment` it
LEFT JOIN `it_built_system_storage_analytics` it2 ON it2.nom_de_baie = it.nom_de_baie and it2.nom_de_la_salle = it.nom_de_la_salle
-- where it.status is null
ORDER  BY nom_de_baie DESC

On regroupe tous les extensions sous le même cerveau pour que leur status corresponde au nom du cerveau

In [ ]:
SELECT *,
( CASE 
 	WHEN t1.status is null and t1.fabricant_status is not null then t1.fabricant_status
    WHEN t1.status not in ('OK-CMDB-ASSET-ID','ASSET_ID OK / SHORT_NAME MISMATCH') then t1.fabricant_status
	END) new_status
    ,
( CASE 
 	WHEN t1.status is null and t1.fabricant_status is not null then Concat(t1.fabricant, ' : ', t1.asset_id_chassis)
    WHEN t1.status not in ('OK-CMDB-ASSET-ID','ASSET_ID OK / SHORT_NAME MISMATCH') then Concat(t1.fabricant, ' : ', t1.asset_id_chassis)
	END) new_new_status
FROM(
    SELECT    it.nom_de_baie,
              it.nom_de_la_salle,
              it.status,
              it.asset_id,
              it.nom,
              it.modele,
              it.reperage_id,
              it.fabricant,
              it2.reperage_id reperage_id_chassis,
              it2.nom         nom_chassis,
              it2.modele      modele_chassis,
              it2.fabricant   fabricant_chassis,
              it2.status fabricant_status,
			  it2.asset_id asset_id_chassis
    FROM      `IT_Equipment` it
    LEFT JOIN `it_built_system_NAS` it2
    ON        it2.nom_de_baie = it.nom_de_baie 
    AND       it2.nom_de_la_salle = it.nom_de_la_salle
    AND		  it2.fabricant = it.fabricant
    WHERE it.status is null and (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%' )
    order by  nom_de_baie DESC
    ) t1
     WHERE it.status not in ('OK-CMDB-ASSET-ID','ASSET_ID OK / SHORT_NAME MISMATCH') and (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%' )

# Update

## Match sur les coordonnees

In [ ]:
Ajout des colonnes des coordonnees en salle
UPDATE assets_filtree
SET 
	Rack = SUBSTRING_INDEX(DALLE, '/', 1)  ,
	Slot = SUBSTRING_INDEX(DALLE, '/', -1) ,
	Slot_begin = SUBSTRING_INDEX(SUBSTRING_INDEX(DALLE, '/', -1), '-', 1) ,
	Slot_end = SUBSTRING_INDEX(DALLE, '-', -1)

In [ ]:
41 rows affected. (Query took 51.4745 seconds.)
SELECT it.Nom_de_la_salle, aa.Salle,
it.Nom_de_baie Nom_de_baie_it, aa.Rack,
it.Slot_U Slot_U_it, CONVERT(aa.Slot_Begin, integer) Slot_Begin,
it.Hauteur_U Hauteur_U_it, CONVERT(aa.Slot_End, integer) Slot_End,
CONVERT(aa.Slot_End, integer) - CONVERT(aa.Slot_Begin, integer) delta_slot
FROM `assets_filtree` aa
INNER JOIN 
	`IT_Equipment` it 
ON 
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
AND
	(it.Hauteur_U = CONVERT(aa.Slot_End, integer) or it.Hauteur_U = CONVERT(aa.Slot_End, integer) - CONVERT(aa.Slot_Begin, integer))

In [ ]:
UPDATE `assets_filtree` aa
INNER JOIN 
	`IT_Equipment` it 
ON 
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
SET aa.status = 'OK-POSITION',
	it.status = 'OK-POSITION'

WHERE aa.status is null and it.status is null

## Match sur les coordonnées inversées (en cours)

## **Exemple** :
- IT_Equipment.reperage_ID = 24J/..
- assets_filtree.Dalle = J24:..

In [ ]:
select it.asset_id asset_id_id, it.short_name, it.nom, it.status status_it, it.fabricant, it.modele, it.reperage_ID, it.nom_de_la_salle,
it.Nom_de_baie, SUBSTR(it.Nom_de_baie,length(it.Nom_de_baie),length(it.Nom_de_baie)) rp,
SUBSTR(it.Nom_de_baie,length(it.Nom_de_baie)-1,length(it.Nom_de_baie)) rp2
from `IT_Equipment` it 
INNER JOIN  `assets_filtree` aa

ON 
	it.Nom_de_la_salle = aa.Salle
AND
	(it.Nom_de_baie = aa.Rack or 
     SUBSTR(it.Nom_de_baie,length(it.Nom_de_baie),length(it.Nom_de_baie)) = aa.Rack or
     SUBSTR(it.Nom_de_baie,length(it.Nom_de_baie),length(it.Nom_de_baie)) = aa.Rack)
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
AND
	(it.Hauteur_U = CONVERT(aa.Slot_End, integer) or it.Hauteur_U = CONVERT(aa.Slot_End, integer) - CONVERT(aa.Slot_Begin, integer))

WHERE aa.status is null and it.status is null

In [ ]:
select
(select it.Nom_de_baie REGEXP '%[^0-9]%') rppp,
from `IT_Equipment` it
where STATUS is null
ORDER BY reperage_id asc

## On regroupe tous les extensions sous le même cerveau pour que leur status corresponde au nom du cerveau

In [ ]:
-- ~400 rows
UPDATE `IT_Equipment` it
 LEFT JOIN `it_built_system_NAS` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant
 SET it.status = 
( CASE 
 	WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.asset_id)
    WHEN it.status not in ('OK-CMDB-ASSET-ID','ASSET_ID OK / SHORT_NAME MISMATCH') then Concat(it2.fabricant, ' : ', it2.asset_id)
	END)
  WHERE it.status not in ('OK-CMDB-ASSET-ID','ASSET_ID OK / SHORT_NAME MISMATCH') and (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%' )

In [ ]:
145 ROWS
UPDATE `IT_Equipment` it
LEFT JOIN `it_build_system_HPC` hpc on (LEFT(it.Nom,6)) = hpc.root_ferme
SET it.status = 
				(CASE WHEN it.status not in ('OK-CMDB-ASSET-ID','ASSET_ID OK / SHORT_NAME MISMATCH') THEN Concat('HPC : ', hpc.root_ferme)
                WHEN it.status is null then Concat('HPC : ', hpc.root_ferme)
                WHEN it.status in ('OK-CMDB-ASSET-ID','ASSET_ID OK / SHORT_NAME MISMATCH') THEN it.status
                END )
WHERE it.nom like '%HPC%' 
OR it.long_name like '%HPC%'

In [ ]:
~45 ROWS
UPDATE   `IT_Equipment` it
LEFT JOIN `it_build_system_HANA` han
 ON        han.nom_de_baie = it.nom_de_baie 
 AND       han.nom_de_la_salle = it.nom_de_la_salle
 AND		  han.fabricant = it.fabricant
 
 SET it.status = 
 				(CASE 
                 WHEN it.status is null and han.status is not null then Concat(it.fabricant, ' : ', han.asset_id)
                 WHEN it.status != 'OK-CMDB-ASSET-ID' OR it.status != 'ASSET_ID OK / SHORT_NAME MISMATCH'  then Concat(han.fabricant, ' : ', han.asset_id)
                 END)
                
WHERE (it.nom_de_baie like '%hana%' 
OR it.nom like '%hana%')

In [ ]:
UPDATE `IT_Equipment` it
 LEFT JOIN `it_built_system_NAS` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant
 SET it.status = 
( CASE 
 	WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.asset_id)
    WHEN it.status != 'OK-CMDB-ASSET-ID' OR it.status != 'ASSET_ID OK / SHORT_NAME MISMATCH'  then Concat(it2.fabricant, ' : ', it2.asset_id)
	END)
    
 WHERE it.status is null and (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%' )